<a href="https://colab.research.google.com/github/nova0816/RAG_pipeline/blob/main/RAG_Pipeline_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RAG Pipeline for Car Rental Information

### 1. Objective
This notebook demonstrates a Retrieval Augmented Generation (RAG) pipeline designed to answer questions based on information extracted from a Sixt Rental Information (France) HTML document. It uses LangChain, HuggingFace models (for embeddings and generation), and FAISS for vector storage.

### 2. How to Use
1.  **Install Libraries**: Run the first code cell to install all necessary Python packages.
2.  **Mount Google Drive & Specify File Path**: Ensure your Google Drive is mounted and update the `file_path` variable in the 'FILE INGESTION' section to point to your `Sixt Rental Information France.html` file.
3.  **Run All Cells**: Execute all code cells sequentially from top to bottom.
4.  **Query the Model**: Modify the `query` variable in the 'Final Question and Answer' section and run the cell to get answers from the RAG pipeline.

### 3. Code Structure
The notebook is organized into the following key sections:
*   **Library Installation**: Installs `langchain`, `pypdf`, `sentence-transformers`, `faiss-cpu`, `transformers`, etc.
*   **File Ingestion**: Mounts Google Drive and reads the specified HTML document.
*   **HTML Parsing and Text Linearization**: Converts the HTML content into a structured, readable text format, extracting headings, paragraphs, and converting tables to Markdown.
*   **Chunking and LangChain Document Creation**: Splits the cleaned text into smaller chunks and converts them into LangChain `Document` objects.
*   **Embedding and Vector Store**: Uses `HuggingFaceEmbeddings` (e.g., `BAAI/bge-small-en-v1.5`) to create embeddings from the chunks and stores them in a `FAISS` vector store for efficient retrieval.
*   **RAG Pipeline Definition**: Sets up the RAG chain using LangChain Expression Language (LCEL) with a prompt template and a HuggingFace LLM (initially `google/flan-t5-small`, then `meta-llama/Meta-Llama-3-8B-Instruct`).
*   **Model Testing**: Demonstrates how to query the RAG pipeline and retrieve answers using both a smaller `google/flan-t5-small` model and a more powerful `meta-llama/Meta-Llama-3-8B-Instruct` model (with 4-bit quantization).

In [ ]:
# Import LLM and chain modules
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate # Using ChatPromptTemplate for modern RAG
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import torch # Required for device management

# New imports for LCEL
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# --- GENERATION ---

# Set device to GPU if available, otherwise CPU
device = 0 if torch.cuda.is_available() else -1

# Load a small LLM (FLAN-T5 is a good choice for text2text generation)
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create a text-generation pipeline
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=150,
    device=device # Use GPU if available
)
llm = HuggingFacePipeline(pipeline=pipe)

# Create the RAG chain using LCEL
# Define the prompt template
prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context.
{context}
If there's ambiguity there, encourge the user to be more specific about the questions.
Question: {question}""")

# Construct the RAG chain using LCEL
qa_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

# --- Final Question and Answer ---
#final_query = "How is the cancellation policy?"
result = qa_chain.invoke(query) # LCEL invoke directly with question string

# Post-process the result to remove extraneous Document information
if "), Document(id='" in result:
    result = result.split("'), Document(")[0].strip() + "'"

print(f"\n--- User Query ---\n{query}")
print(f"\n--- Generated Answer ---\n{result}")

In [ ]:
# 1. Install Libraries (Run this cell in Colab)
!pip install --upgrade langchain pypdf sentence-transformers faiss-cpu transformers langchain-text-splitters langchain-huggingface langchain-community
# Install the necessary libraries
!pip install -q transformers accelerate bitsandbytes sentence-transformers langchain torch
!pip install -U bitsandbytes

In [ ]:
# Install libraries (Run this in your Colab notebook)
!pip install beautifulsoup4 pandas html5lib langchain

Read file

In [ ]:
from google.colab import drive

# Mount Google Drive to the default mount point /content/drive
drive.mount('/content/drive')

print("Google Drive mounted successfully.")

In [ ]:
# Install libraries (Run this cell in Colab)
#!pip install beautifulsoup4 pandas html5lib langchain

# Import necessary modules
import pandas as pd
from bs4 import BeautifulSoup, Tag
from io import StringIO
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os


# --- FILE INGESTION (Direct Drive Read) ---

# **IMPORTANT**: Define the full path to your file based on the screenshot structure
file_path = "/content/drive/MyDrive/Colab Notebooks/RAG Material/Sixt Rental Information France.html"
html_content = ""

# Check for the file and read its content
if os.path.exists(file_path):
    try:
        # Read the file content.
        # Using 'windows-1252' encoding as suggested for this file type
        with open(file_path, 'r', encoding='windows-1252') as f:
            html_content = f.read()

        print(f"File '{file_path}' successfully loaded.")
        print(f"Size of loaded HTML content: {len(html_content)} characters.")

    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        html_content = "<html><body>Error reading content.</body></html>"
else:
    print(f"Error: File not found at the expected path: {file_path}")
    print("Please ensure the Drive is mounted and the path is exactly correct.")
    html_content = "<html><body>File is missing.</body></html>"


print("\n--- Setup Complete ---")

In [ ]:
html_content

Read Html Document

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup, Tag
from io import StringIO
import os
from google.colab import drive


## 2. Table Extraction and Text Linearization (FIXED)

def parse_html_for_rag(html_content):
    """
    Parses HTML by finding all structural elements (headings, paragraphs, tables)
    within the main content div, converts tables to Markdown, and cleans text.

    The fix targets the inner <div> blocks that contain the actual content structure.
    """
    if not html_content:
        return ""

    soup = BeautifulSoup(html_content, 'html.parser')

    # Target the main content area
    content_div = soup.find('div', id='sx-gc-content')
    if not content_div:
        return "Error: Could not find main content div (sx-gc-content)."

    processed_text_segments = []

    # Process the header text right after the main div start, but before the inner divs
    header_text = content_div.find('h3', class_='sx-terms-header-selection')
    if header_text:
        processed_text_segments.append(f"\n# {header_text.get_text(strip=True)}\n")


    # The actual content is nested inside subsequent sibling <div> tags
    # We look for all immediate child <div> tags that contain the sections
    content_sections = content_div.find_all('div', recursive=False)

    # Iterate through each section div found
    for section in content_sections:

        # Iterate through all children of the inner section to maintain order
        for element in section.children:
            if isinstance(element, Tag):

                # Handle Tables: Extract and convert to Markdown
                if element.name == 'table':
                    try:
                        # Use pandas to extract the table data
                        dfs = pd.read_html(StringIO(str(element)), flavor='bs4')
                        if dfs:
                            df = dfs[0]
                            # Convert the DataFrame to a Markdown string
                            markdown_table = df.to_markdown(index=False)

                            processed_text_segments.append(
                                f"\n\n--- Start Table ---\n{markdown_table}\n--- End Table ---\n\n"
                            )
                    except ValueError:
                        processed_text_segments.append(f"\n\n[Could not parse complex table]\n\n")

                # Handle Headings and Paragraphs: Extract text
                elif element.name in ['h2', 'h3', 'p']:
                    text = element.get_text(strip=True)
                    # Add separators for better readability and chunking context
                    if element.name == 'h2':
                        text = f"\n\n## {text}"
                    elif element.name == 'h3':
                        text = f"\n### {text}"

                    # Also include any links' text for context, but not the link itself
                    if element.name == 'p':
                        # Find all <a> tags and extract their text
                        for a in element.find_all('a'):
                             a_text = a.get_text(strip=True)
                             if a_text not in text: # Avoid double counting text already captured by parent
                                 text += f" ({a_text})"

                    processed_text_segments.append(text)

    # Combine all segments into one large, structured string
    full_cleaned_text = "\n".join(processed_text_segments).strip()

    return full_cleaned_text

# Execute the parsing function with the loaded content
cleaned_rag_text = parse_html_for_rag(html_content)

print(f"\n--- Parsing Results ---")
print(f"Total characters in cleaned text: {len(cleaned_rag_text)}")
print("\n--- Example of Cleaned Text (First 1500 chars) ---")
print(cleaned_rag_text[:1500])
print("\n--- Parsing and Linearization Complete ---")


## 3. Chunking and LangChain Document Creation

# Convert the cleaned text into a single LangChain Document
document = Document(page_content=cleaned_rag_text, metadata={"source": "Sixt Rental Info France"})

# Define the text splitter for RAG chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# Split the document into chunks for embedding
final_chunks = text_splitter.split_documents([document])

print(f"\nTotal text chunks created: {len(final_chunks)}")
print("\n--- Example Chunk Content with Table Data ---")
# Print the first chunk to show structure
print(final_chunks[0].page_content)
print("\n--- Another Chunk (Searching for a table) ---")
# Find a chunk that contains a table for demonstration
for chunk in final_chunks:
    if 'Start Table' in chunk.page_content:
        print(chunk.page_content)
        break

In [ ]:
# Convert the cleaned text into a single LangChain Document
document = Document(page_content=cleaned_rag_text, metadata={"source": "Sixt Rental Info France"})

# Define the text splitter for RAG chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# Split the document into chunks for embedding
final_chunks = text_splitter.split_documents([document])

print(f"\nTotal text chunks created: {len(final_chunks)}")
print("\n--- Example Chunk Content with Table Data ---")
# Find a chunk that contains a table for demonstration
for chunk in final_chunks:
    if 'Start Table' in chunk.page_content:
        print(chunk.page_content)
        break
# If no table found in the first few chunks, print the first chunk
else:
    print(final_chunks[0].page_content)

RAG Pipeline

In [ ]:
# Import necessary modules
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document # Used to convert text to LangChain Document format

# Import embedding and vector store modules
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


In [ ]:
# --- EMBEDDING & RETRIEVAL (Vector Store) ---

# Choose an open-source embedding model
embedding_model_name = "BAAI/bge-small-en-v1.5"
#sentence-transformers/all-MiniLM-L6-v2

embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Create a FAISS vector store from the chunks
vector_store = FAISS.from_documents(final_chunks, embeddings)

print("Vector Store created and documents embedded.")
print("--- Embedding and Indexing Complete ---")


In [ ]:

# Define a retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 10}) # Retrieve top 2 chunks

# Example Retrieval
query = " I just booked and want to cancel, can I get a refund?"
retrieved_docs = retriever.invoke(query)

print(f"\n--- Retrieved Context (Top {len(retrieved_docs)} Chunks) ---")
for i, doc in enumerate(retrieved_docs):
    print(f"Chunk {i+1}: {doc.page_content}...")

Test model 1: Generate answer by model google/flan-t5-small, a text model

Test Model 2: Generate model by Meta-Llama-3-8B-Instruct

In [ ]:


# Import necessary modules
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch

# --- Model Selection and Configuration ---

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# Alternatively, use a smaller but capable model:
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Check for GPU and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Configuration for 4-bit quantization (REQUIRED to fit 8B/7B models on Colab T4/P100)
# This reduces memory footprint by loading the model in 4-bit precision.
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# Install huggingface_hub if not already installed
!pip install -q huggingface_hub

In [ ]:
print(f"Loading model: {model_name}...")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # Set pad token for batching (best practice)

# Load Model with Quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.bfloat16, # Use bfloat16 for computation
)

# Create the text-generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,  # Set a higher limit for comprehensive answers
    temperature=0.1,     # Low temperature for factual RAG answers
    do_sample=True,      # Enable sampling for varied responses
    top_p=0.95,
    repetition_penalty=1.1,
)

# Wrap the pipeline in a LangChain LLM object
llm = HuggingFacePipeline(pipeline=pipe)

print("LLM and Pipeline Loaded Successfully.")

In [ ]:
from huggingface_hub import login
from google.colab import userdata

# Retrieve the token from Colab secrets
hf_token = userdata.get('HF_TOKEN')

# Authenticate with Hugging Face
login(token=hf_token)

print("Hugging Face authentication complete. You can now try loading the gated model.")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# Define the prompt template
prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context.
{context}
If there's ambiguity there, encourge the user to be more specific about the questions.
Also, please quote the title of the section where the answer is found.
Question: {question}""")

# Construct the RAG chain using LCEL
qa_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

# Example Query
result = qa_chain.invoke(query) # LCEL invoke directly with question string

# Post-process the result to remove extraneous Document information if any
# This specific post-processing might be less relevant for text-generation LLMs but kept for consistency.
if "), Document(id='" in result:
    result = result.split("')', Document(")[0].strip() + "'"

print(f"\n--- User Query ---\n{query}")
print(f"\n--- Generated Answer ---\n{result}")